In [6]:
import os
import re
import pandas as pd

def preprocess_text(text):
    """
    预处理文本数据
    - 移除去识别括号
    - 移除编号列表
    - 替换缩写
    - 移除特定关键词
    - 移除重复字符
    """
    text = re.sub(r'\[(.*?)\]', '', text)  # 移除去识别括号
    text = re.sub(r'[0-9]+\.', '', text)  # 移除编号列表
    text = re.sub(r'dr\.', 'doctor', text)  # 替换缩写
    text = re.sub(r'm\.d\.', 'md', text)  # 替换缩写
    text = re.sub(r'admission date:', '', text)  # 移除特定关键词
    text = re.sub(r'discharge date:', '', text)  # 移除特定关键词
    text = re.sub(r'--|__|==', '', text)  # 移除重复字符
    return text

def preprocessing(df_notes):
    """
    预处理数据框中的文本数据
    - 填充缺失值
    - 移除换行符和回车符
    - 去除首尾空格
    - 转换为小写
    - 应用自定义预处理函数
    - 移除多余的空白
    """
    df_notes['TEXT'] = df_notes['TEXT'].fillna(' ')  # 填充缺失值
    df_notes['TEXT'] = df_notes['TEXT'].str.replace('\n', ' ')  # 移除换行符
    df_notes['TEXT'] = df_notes['TEXT'].str.replace('\r', ' ')  # 移除回车符
    df_notes['TEXT'] = df_notes['TEXT'].apply(str.strip)  # 去除首尾空格
    df_notes['TEXT'] = df_notes['TEXT'].str.lower()  # 转换为小写
    df_notes['TEXT'] = df_notes['TEXT'].apply(preprocess_text)  # 应用自定义预处理函数
    df_notes['TEXT'] = df_notes['TEXT'].apply(lambda x: " ".join(x.split()))  # 移除多余的空白
    return df_notes

def save_text_chunks(texts, save_path, chunk_size=10):
    """
    将文本数据按块保存到指定路径
    - texts: 文本列表
    - save_path: 保存路径
    - chunk_size: 每个文件包含的文本块数量
    """
    if not os.path.exists(save_path):
        os.makedirs(save_path)  # 如果路径不存在，创建路径

    for i in range(0, len(texts), chunk_size):
        chunk = texts[i:i + chunk_size]  # 获取当前块
        chunk_text = "\n".join(chunk)  # 将块内文本合并为一个字符串
        file_name = os.path.join(save_path, f'text_chunk_{i // chunk_size + 1}.txt')  # 生成文件名
        with open(file_name, 'w', encoding='utf-8') as f:
            f.write(chunk_text)  # 将文本写入文件




In [2]:
# 定义CSV文件路径
file_path ='NOTEEVENTS.csv'
column_name = 'CATEGORY'

# 初始化空的Series以存储聚合计数
aggregated_counts = pd.Series(dtype=int)





In [7]:

# 设置块大小
chunk_size = 10000  # 根据系统内存容量调整

# 读取CSV文件并进行分块处理
for chunk in pd.read_csv(file_path, chunksize=chunk_size, usecols=[column_name]):
    # 计算当前块的值计数并添加到聚合计数中
    aggregated_counts = aggregated_counts.add(chunk[column_name].value_counts(), fill_value=0)

# 将聚合计数转换为整数
aggregated_counts = aggregated_counts.astype(int)

# 打印聚合计数结果
print(aggregated_counts)


CATEGORY
Case Management         1934
Consult                  196
Discharge summary     119304
ECG                   418102
Echo                   91588
General                16602
Nursing               447112
Nursing/other        1644994
Nutrition              18836
Pharmacy                 206
Physician             283248
Radiology            1044558
Rehab Services         10862
Respiratory            63478
Social Work             5340
dtype: int64


In [9]:


# 初始化一个空列表以存储过滤后的DataFrame
filtered_chunks = []

# 读取CSV文件并进行分块处理
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # 过滤出类别为'Social Work'的记录
    filtered_chunk = chunk[chunk['CATEGORY'] == 'Social Work']
    filtered_chunks.append(filtered_chunk)

# 将所有过滤后的块连接成一个DataFrame
soc = pd.concat(filtered_chunks, ignore_index=True)

# 应用预处理函数
soc = preprocessing(soc)

# 标记包含'discharge'的记录
soc['discharge'] = soc['TEXT'].str.contains('discharge')

# 提取包含'discharge'的文本记录
discharge_notes = soc[soc['discharge']]['TEXT'].tolist()

# 输出文件夹路径
output_dir = 'discharge'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 将每十行文本保存为一个单独的txt文件
save_text_chunks(discharge_notes, output_dir, chunk_size=5)

print(f"Saved {len(discharge_notes)//5 + 1} files in the {output_dir} directory.")

Saved 25 files in the discharge directory.
